In [1]:
#import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import os
import sys
import matplotlib.pyplot as plt

In [2]:
deeplabv3plus_dir="./src"
sys.path.append(deeplabv3plus_dir)

In [3]:
gpu_options = tf.compat.v1.GPUOptions(visible_device_list="1", allow_growth=True)
config = tf.compat.v1.ConfigProto(gpu_options = gpu_options)
tf.compat.v1.enable_eager_execution(config=config)

In [4]:
from model import deeplab_v3plus_transfer_os16
#from image_utils import make_x_from_image_paths,make_y_from_image_paths,convert_y_to_image_array
from data_gen import DataGenerator

In [5]:
import importlib
importlib.reload(sys.modules['model'])

<module 'model' from './src/model.py'>

In [6]:
out_dir = "xception_transfer_os16"
os.makedirs(out_dir, exist_ok=True)

In [7]:
n_categories = 21
image_size = (512,512)

In [8]:
seg_img_dir = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/SegmentationClass"
img_dir = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/JPEGImages"
train_set_path = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt"
valid_set_path = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt"

with open(train_set_path) as f:
    train_img_names = f.read().split("\n")[:-1]
with open(valid_set_path) as f:
    valid_img_names = f.read().split("\n")[:-1]

img_paths = [os.path.join(img_dir,train_img_names[i]) + ".jpg" for i in range(len(train_img_names))]
seg_img_paths = [os.path.join(seg_img_dir,train_img_names[i]) + ".png" for i in range(len(train_img_names))]

valid_x_paths = [os.path.join(img_dir,valid_img_names[i]) + ".jpg" for i in range(len(valid_img_names))]
valid_y_paths = [os.path.join(seg_img_dir,valid_img_names[i]) + ".png" for i in range(len(valid_img_names))]

In [9]:
batch_size=16
n_epochs=300

In [10]:
train_data_gen = DataGenerator(n_categories, image_size, batch_size, img_paths    , seg_img_paths, augmentation=True )
valid_data_gen = DataGenerator(n_categories, image_size, batch_size, valid_x_paths, valid_y_paths, augmentation=False)

In [11]:
encoder = keras.applications.Xception(input_shape=(512,512,3), weights="imagenet", include_top=False)
layer_name_to_decoder = "block4_sepconv1_act"
encoder_end_layer_name = "block13_sepconv2_bn"
model = deeplab_v3plus_transfer_os16(n_categories, encoder, layer_name_to_decoder, encoder_end_layer_name)

In [ ]:
#model.summary()

In [13]:
loss_function = tf.keras.losses.categorical_crossentropy
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt, loss=loss_function, metrics=["accuracy"])

In [14]:
filepath = os.path.join(out_dir,'{epoch:06d}.h5')
cp_cb = keras.callbacks.ModelCheckpoint(filepath, 
                                        monitor='val_loss', 
                                        verbose=0, 
                                        save_best_only=True, 
                                        save_weights_only=False, 
                                        mode='auto')

In [ ]:
#hist = model.fit_generator(data_gen, validation_data=(valid_x, valid_y), epochs=par.n_epochs, steps_per_epoch=par.n_batch, callbacks=[cp_cb])
hist = model.fit_generator(train_data_gen,
                           epochs=n_epochs,
                           steps_per_epoch=len(train_data_gen),
                           validation_data=valid_data_gen,
                           validation_steps=len(valid_data_gen),
                           shuffle = True,
                           workers=8,
                           use_multiprocessing=True,
                           callbacks=[cp_cb])
#hist = model.fit_generator(data_gen, epochs=par.n_epochs, steps_per_epoch=par.n_batch, workers=8, use_multiprocessing=True)

Epoch 1/300


W1128 14:03:25.424393 140735685348560 deprecation.py:323] From /home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1394: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


92/92 [==============================] - 199s 2s/step - loss: 2.3942 - acc: 0.6051 - val_loss: 2.6726 - val_acc: 0.7528
Epoch 2/300
92/92 [==============================] - 179s 2s/step - loss: 2.0278 - acc: 0.7350 - val_loss: 2.3336 - val_acc: 0.7495
Epoch 3/300
92/92 [==============================] - 180s 2s/step - loss: 1.8389 - acc: 0.7617 - val_loss: 2.0137 - val_acc: 0.7770
Epoch 4/300
92/92 [==============================] - 188s 2s/step - loss: 1.6784 - acc: 0.7840 - val_loss: 1.6405 - val_acc: 0.8576
Epoch 5/300
92/92 [==============================] - 174s 2s/step - loss: 1.5392 - acc: 0.8017 - val_loss: 1.3767 - val_acc: 0.8794
Epoch 6/300
92/92 [==============================] - 297s 3s/step - loss: 1.4111 - acc: 0.8177 - val_loss: 1.1939 - val_acc: 0.8799
Epoch 7/300
92/92 [==============================] - 183s 2s/step - loss: 1.2945 - acc: 0.8333 - val_loss: 1.0686 - val_acc: 0.8802
Epoch 8/300
92/92 [==============================] - 191s 2s/step - loss: 1.2016 - acc: 

In [ ]:
plt.figure(figsize=(30,10))

plt.subplot(1,3,1)
plt.plot(hist.history["loss"], label="loss")
plt.plot(hist.history["val_loss"], label="val_loss")
plt.legend()

plt.subplot(1,3,2)
plt.plot(hist.history["acc"], label="acc")
plt.plot(hist.history["val_acc"], label="val_acc")
plt.legend()
plt.savefig(os.path.join(out_dir,'losscurve.png'))

In [ ]:
model.save(os.path.join(out_dir,'final_epoch.h5'))